# SpeechAnalyzer

## Progetto Technologies for Advanced Programming 2019/2020

- Aldo Fiorito X81000447
- Francesco Petrosino X81000533  
Studenti presso Università di Catania cdl Informatica

## Idea e scopo di SpeechAnalyzer
L'idea è nata osservando una community di persone all'interno dell'applicativo VOIP per le chat vocali Discord. 
<br \>
La domanda principale posta è stata "Chi è il più chiacchierone del gruppo?"

Bisogna fare prima una premessa ...

![](train.jpg)

## Da qui SpeechAnalyzer

Fornisce statistiche e metriche su numerosi flussi vocali all'interno della propria rete

## Architettura dell'applicativo

![](schema.png)

### In breve
1. L'utente si registra all'applicativo
2. Un file json costruito dal suo flusso vocale viene mandato a un kafka-producer, avente come topic un campo   specificato dall'user
3. Spark , essendo anche kafka-consumer , processa ed elabora altre informazioni dal dato e costruisce un datafraframe attraverso sparksql 
4. La nuova struttura creata viene indicizzata su elasticsearch
5. Kibana si occupa di ottenere le metriche e le statistiche dai dati indicizzati attraverso un'interfaccia user-end

## Flusso operativo

L'utente,tramite una form, viene fatto registrare all'applicativo inserendo il proprio nome,la propria compagnia( che sarà il topic kafka) e la propria lingua d'appartenenza.  
**Perchè la lingua?**
Perchè l'analizzatore riesce ad elaborare qualiasi lingua ( al momento le 5 principali ).  
Ogni input verrà tradotto nella stessa lingua

### PreProcessing del dato
Un primo step di preprocessing dell'input vocale è quello di filtraggio del testo.       \
Attraverso un file in locale di "badwords" verranno sostituite quelle parole considerate blasfeme o offensive
Come secondo step il testo filtrato viene tradotto in inglese


### Kafka
E' la prima tecnologia della pipeline che andremo ad utilizzare
( spiegare kafka e zookeaper )






Il topic-kafka verrà creato da codice python:

admin_client = KafkaAdminClient(
                   bootstrap_servers="192.168.1.28:9092", 
                   client_id='test'
               ) <br />
topic_list = [NewTopic(name=topicKafka, num_partitions=1, replication_factor=1)]
admin_client.create_topics(new_topics=topic_list, validate_only=False)

Il file json inviato al kafka-producer avrà questa struttura :\\ 

{"name":"saro" , "message" : "Microsoft has released security updates ","topic" : "Tech","language" : "en","id":"75df85bf"}

Dove topic si riferisce al predict ottenuto dall'algoritmo di kmeans implementato.

### Modello Machine Learning e K-means

![](ml.png)

L'immagine appena mostrata rappresenta il modello di kmeans utilizzato e salvato in locale attraverso la libreria "pickle" <br />
Per inizializzare il modello è stato dato in pasto un dataset di news ottenuto al seguente indirizzo URL
http://mlg.ucd.ie/datasets/bbc.html. <br />
Le news appartengono a 5 categorie diverse: business, entertainment, politics, sport, tech.
Questi documenti rawText devono essere processati prima di essere passati all'algoritmo.
StopWords,rimuovere le parole insignificanti
Stemming,portare le parole alla root form
tokenization , per

### Salvataggio e restore del modello ML
k_model=MLAPI()
Pkl_Filename = "kmodel.pkl"
    with open(os.path.join('./python/bin/MachineL',Pkl_Filename), 'wb') as file:  
        pickle.dump(k_model,file)
        
with open('./python/bin/MachineL/kmodel.pkl', 'rb') as f:
        model = pickle.load(f)      

Questo permette una rapida esecuzione dell'algoritmo, in quanto l'oggetto viene semplicemente caricato all'avvio e non inizializzato di nuovo.
Il predict del topic del kmeans viene effettuato solamente attraverso una semplice chiamata a metodo \
number = model.getPrediction(translated.text)

### Kafka Producer

producer = KafkaProducer(bootstrap_servers=['192.168.1.28:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8')
                        )    <br />
producer.send(topicKafka,value=data)   <br />

L'indirizzo ip del boostrap_servers l'indirizzo ip statico settato a inizio per kafkaServer

### Spark 
Spiegazione spark


Avrà anche il compito di essere un kafka consumer <br \>
E' stato scelto di usare il classico approccio **Receiver-based Approach** in quanto l'approccio diretto ( essendo scritto in Python , non è disponibile)

KafkaUtils.createStream(ssc, zkQuorum, "spark-streaming-consumer", {topic: 1},)

Dove ssc è lo streaming context di spark con intervallo microbatch di 2 secondi
zkQuorum ,indirizzo più porta dove gira zookeaper , nel nostro caso "127.0.0.1:2181"
Un nome per il consumer group id
Quanti topic per kafkapartitions leggere

#### Ricezione e preparazione della struttura

Viene definito uno schema per accogliere il json :

In [ ]:
schema = StructType([StructField("id",StringType(),True),StructField("name", StringType(), True),StructField("message", StringType(), True),StructField("topic", StringType(),True),StructField("language",StringType(),True)])
storage = sqlContext.createDataFrame(sc.emptyRDD(), schema)

Successivamente si è passati alle funzioni lambda per estrapolare i dati al fin di calcolare metriche e statistiche.

![](lambda.jpg)

Il tutto viene inserito in un dataframe a cui,a fine riempimento,viene applicata una funzione lambda

appendend.rdd.map(lambda item: {'timestamp': milli ,'id': item['id'],'name': item['name'],'message': item['message'],'profanity_count': item['profanity_count'],'words_count': item['word_count'],'language': item['language'],'topic': item['topic']})

final_rdd = new.map(json.dumps).map(lambda x: ('key', x))
<br \>

RDD finale viene convertito in json per permettere a elastic search di indicizzare i dati nel database
E' importante ricordare di aggiungere un timestamp alla struttura per permettere la futura gestione e visualizzazione su Kibana

### ElasticSearch

Elasticsearch è un server di ricerca con supporto ad architetture distribuite. Utilizza un'interfaccia RESTful, mentre le informazioni sono gestite come documenti JSON.

Indici e documenti

#### Setting su Python

elastic_host="localhost"
elastic_index="users"
elastic_document="_doc"

es_write_conf = {
"es.nodes" : elastic_host,
"es.port" : '9200',
"es.resource" : '%s/%s' % (elastic_index,elastic_document),
"es.input.json" : "yes"
}

conf = SparkConf(loadDefaults=False)
conf.set("es.index.auto.create", "true")

#### Creazione indice
Tutti i json creati in spark convergono verso un unico indice chiamato "users".

elastic = Elasticsearch(hosts=[elastic_host])

response = elastic.indices.create(
    index=elastic_index,
    body=mapping,
    ignore=400 # ignore 400 already exists code
)

#### Salvataggio su ES

final_rdd.saveAsNewAPIHadoopFile(
    path='-',
    outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
    keyClass="org.apache.hadoop.io.NullWritable",
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
    conf=es_write_conf)

#### Verifica se sono stati creati i documenti

Comando query

E' possibile cancellare l'index attraverso il seguente comando

http://localhost:9200/[index_here]



### KIbana

Kibana è una dashboard di visualizzazione dati open source per Elasticsearch. <br />
Fornisce funzionalità di visualizzazione in cima al contenuto indicizzato su un cluster Elasticsearch
La dashboard è user-friendly e personalizzabile.

#### Creazione indice

Prima di poter iniziare e visualizzare i dati è necessario creare e definire un "index pattern"

foto

#### Selezione del timestamp field
Per poter tracciare i dati indicizzati bisogna specificare quel campo timestamp inserito nel file json creato su spark.

foto

### DEMO LIVE 
Passiamo alla demo live, ma prima,
un cattivo uso di "SpeechAnalyzer"...

![](spark.jpg)